# Pipeline de Emisiones RM — Resumen de Pasos

Este cuaderno resume los scripts necesarios para reconstruir y enriquecer las emisiones RETC de la Región Metropolitana.

## 1. Normalización de datos RAW

```bash
python codigo/src/convertir_raw_a_csv_por_ano.py \
  --indir data/raw/descargas_retc \
  --outdir data/interim/01_emisiones_por_ano
```
- Convierte archivos originales a CSV por año.
- Ajusta separador `;`, decimales `.` y agrega fila de tipos (numerico/texto).
- Fija la columna `unidad` en `t/año`.


## 2. Filtrado RM

```bash
python codigo/src/filtrar_region_metropolitana.py \
  --indir data/interim/01_emisiones_por_ano \
  --outdir data/interim/02_emisiones_por_ano_rm
```
- Detecta la columna región y conserva sólo la RM (incluye alias).


## 3. Fusión por tramos

```bash
python codigo/src/fusionar_emisiones_por_grupo.py \
  --indir data/interim/02_emisiones_por_ano_rm \
  --outdir data/interim/03_emisiones_rm_fusionadas
```
- Fusiona 2005-2015 y 2016-2018 en tablas únicas, copia años restantes.


### 3.1 Enriquecimiento

- Contaminantes: `python codigo/src/estandarizar_contaminantes_rm.py --indir data/interim/03_emisiones_rm_fusionadas --outdir data/interim/03_emisiones_rm_fusionadas`
- Actividades y macro-categorías: `python codigo/src/estandarizar_ciiu_rm.py --indir data/interim/03_emisiones_rm_fusionadas --outdir data/interim/03_emisiones_rm_fusionadas`.
- Emisión total: `python codigo/src/agregar_emision_total_rm.py --indir data/interim/03_emisiones_rm_fusionadas`.
- ID único: `python codigo/src/agregar_id_unico_rm.py --indir data/interim/03_emisiones_rm_fusionadas`.


### 3.2 Tabla consolidada y geografía

- Consolidar tablas RM: `python codigo/src/fusionar_emisiones_consolidadas.py --indir data/interim/03_emisiones_rm_fusionadas --outdir data/interim/04_emisiones_consolidadas`.
- Completar coordenadas: `python codigo/src/completar_coordenadas_con_centros.py --centros data/raw/comunas/comunas_rm_centros.csv --consolidado data/interim/04_emisiones_consolidadas/retc_RM_consolidado.csv`.
- Unidades del paisaje: `python codigo/src/asignar_unidad_paisaje_rm.py --consolidado data/interim/04_emisiones_consolidadas/retc_RM_consolidado.csv --poligonos geo/insumos/UnidadesPaisajeRM/unidades-paisaje-V1.gpkg`.


## 4. Productos analíticos

- Resumen anual por contaminante: `python codigo/src/graficar_totales_por_variable.py --indir data/interim/emisiones_por_variable_fusionadas --outdir outputs/graficos/emisiones_totales`.
- Distribución de emisiones (histogramas): `python codigo/src/graficar_distribucion_emisiones_por_contaminante.py --input data/interim/04_emisiones_consolidadas/retc_RM_consolidado.csv --outdir outputs/graficos/emisiones_distribucion`.


## 5. Referencias

- Catálogos en `metadata/` (contaminantes, CIIU, glosario).
- Consolidado final: `data/interim/04_emisiones_consolidadas/retc_RM_consolidado.csv`.
